[Reference](https://medium.com/@marcnealer/fastapi-http-authentication-f1bb2e8c3433)

# Creating a settings module

In [1]:
from dotenv import find_dotenv, dotenv_values
import pathlib
import string


file_path = pathlib.Path().cwd()
static_dir = str(pathlib.Path(pathlib.Path().cwd(), "static"))
config = dotenv_values(find_dotenv(".test_fastapi_config.env"))

db_user = config.get("DB_USER")
db_password = config.get("DB_PASSWORD")
db_host = config.get("DB_HOST")
db_name = config.get("DB_NAME")
db_port = config.get("DB_PORT")
db_url = f"asyncpg://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}"
db_modules = {"models": ["models"]}

max_age = 3600

session_choices = string.ascii_letters + string.digits + "=+%$#"

# Database models and set-up

In [2]:
from tortoise import fields, models
from tortoise.contrib.pydantic import pydantic_model_creator


def default_scope():
    return ["authenticated"]


class Users(models.Model):
    """
    The User model
    """

    id = fields.IntField(primary_key=True)
    username = fields.CharField(max_length=20, unique=True)
    first_name = fields.CharField(max_length=50, null=True)
    last_name = fields.CharField(max_length=50, null=True)
    p_hash = fields.BinaryField(max_length=128, null=True)
    p_salt = fields.BinaryField(max_length=128, null=True)
    scope = fields.JSONField(default=default_scope)
    info = fields.JSONField(default=dict)

    class PydanticMeta:
        exclude = ["password_hash"]


User_Pydantic = pydantic_model_creator(Users, name="User")
UserIn_Pydantic = pydantic_model_creator(Users, name="UserIn", exclude_readonly=True)


class Session(models.Model):

    id = fields.IntField(primary_key=True)
    token = fields.CharField(max_length=128, unique=True, db_index=True)
    user = fields.IntField(default=0)
    created_at = fields.DatetimeField(auto_now_add=True)
    expires_at = fields.DatetimeField(auto_now_add=True)


Session_Pydantic = pydantic_model_creator(Session, name="Session")

# Adding the ORM to FastAPI

In [3]:
@app.on_event("startup")
async def my_event():
  pass

In [4]:
from fastapi import FastAPI
import settings
from contextlib import asynccontextmanager
from typing import AsyncGenerator
from tortoise.contrib.fastapi import RegisterTortoise


@asynccontextmanager
async def lifespan(app: FastAPI) -> AsyncGenerator[None, None]:
    # app startup
    async with RegisterTortoise(
        app,
        db_url=settings.db_url,
        modules=settings.db_modules,
        generate_schemas=True,
        add_exception_handlers=True,
    ):
        # db connected
        yield
        # app teardown

main_app = FastAPI(lifespan=lifespan)

# Middleware

In [5]:
@main_app.middleware("http")
async def session_middleware(request: Request, call_next):
    cookie_val = request.cookies.get("session")
    if cookie_val:
        request.scope['session'] = cookie_val
    else:
        request.scope['session'] = "".join(random.choices(settings.session_choices, k=128))
    response = await call_next(request)
    response.set_cookie("session", value=request.session,
                        max_age=settings.max_age, httponly=True)
    return response

In [6]:
from models import Session, Users


class BaseUser:
    @property
    def is_authenticated(self) -> bool:
        raise NotImplementedError()


class UnauthenticatedUser(BaseUser):
    @property
    def is_authenticated(self) -> bool:
        return False


class AuthUser(BaseUser):
    def __init__(self, session: Session) -> None:
        self.session = session
        self.__user = None

    @property
    def is_authenticated(self) -> bool:
        return True

    async def user(self) -> Users:
        if not self.__user:
            self.__user = await Users.get_or_none(id=self.session.user)
        return self.__user

In [7]:
@main_app.middleware("http")
async def authentication_middleware(request: Request, call_next):
    token = request.cookies.get("session")
    if not token:
        request.scope["auth"] = ["anonymous"]
        request.scope["user"] = UnauthenticatedUser()
    else:
        session = await Session.get_or_none(token=token)
        if session is None:
            request.scope["auth"] = ["anonymous"]
            request.scope["user"] = UnauthenticatedUser()
        else:
            request.scope["user"] = AuthUser(session)
            user = await Users.get_or_none(id=session.user)
            request.scope["auth"] = user.scope
    response = await call_next(request)
    return response

In [8]:
async def authenticate(request: Request, user: str, password: str) -> bool:
    user = await Users.get_or_none(username=user)
    if user:
        p_hash = hashlib.pbkdf2_hmac("sha256", password.encode(), user.p_salt, 100000)
        if p_hash == user.p_hash:
            expires = datetime.datetime.now() + datetime.timedelta(seconds=max_age)
            if await Session.filter(token=request.session).exists():
                await Session.filter(token=request.session).delete()
            session = await Session.create(user=user.id, expires_at=expires,
                                           scope=user.scope, token=request.session)
            request.scope["session"] = session.token
            request.scope["auth"] = user.scope
            auth_user = AuthUser(session)
            auth_user.__user = user
            request.scope["user"] = auth_user
            return True
        else:
            return False
    else:
        return False

In [9]:
from auth.exceptions import NotLoggedInException, PermissionFailedException, AlreadyLoggedInException
from fastapi import Request, BackgroundTasks
import datetime
from settings import max_age
from models import Session


async def update_session_expiry(request: Request):
    session = await Session.get(token=request.session)
    session.expires_at = datetime.datetime.now() + datetime.timedelta(seconds=max_age)
    await session.save()
    await Session.filter(expires_at__lt=datetime.datetime.now()).delete()


async def already_logged_in(request: Request):
    if request.user.is_authenticated:
        raise AlreadyLoggedInException()
    return True


async def logged_in(request: Request, background_tasks: BackgroundTasks) -> bool:
    if not request.user.is_authenticated:
        raise NotLoggedInException()
    background_tasks.add_task(update_session_expiry, request)
    return True


async def admin_user(request: Request, background_tasks: BackgroundTasks) -> bool:
    if not request.user.is_authenticated:
        raise NotLoggedInException()
    if "admin" not in request.auth or "super" not in request.auth:
        raise PermissionFailedException("You need to be marked as an admin user to access this endpoint")
    background_tasks.add_task(update_session_expiry, request)
    return True


async def super_user(request: Request, background_tasks: BackgroundTasks) -> bool:
    if not request.user.is_authenticated:
        raise NotLoggedInException()
    if "super" not in request.auth:
        raise PermissionFailedException("You need to be marked as an admin user to access this endpoint")
    background_tasks.add_task(update_session_expiry, request)
    return True

In [10]:
from fastapi.responses import RedirectResponse, HTMLResponse
from fastapi import Request, FastAPI

"""
Authentication custom exceptions and exception handlers
"""


class NotLoggedInException(Exception):
    def __init__(self):
        pass


class AlreadyLoggedInException(Exception):
    def __init__(self):
        pass


def already_logged_in_handler(request: Request, exc: AlreadyLoggedInException):
    return RedirectResponse("/")


def not_logged_in_handler(request: Request, exc: NotLoggedInException):
    """Redirect to the login page if login is required"""
    return RedirectResponse("/auth")


class PermissionFailedException(Exception):
    def __init__(self, permissions: list):
        self.permissions = permissions


def permission_failed_handler(request: Request, exc: PermissionFailedException):
    """shows an error page if the users authentication scope fails to meet the requirements"""
    return HTMLResponse(content="templates/permission_failed.html", status_code=401)


def auth_exceptions_add(app: FastAPI):
    """Loads the exception handlers into the app"""
    app.add_exception_handler(NotLoggedInException, not_logged_in_handler)
    app.add_exception_handler(PermissionFailedException, permission_failed_handler)
    app.add_exception_handler(AlreadyLoggedInException, already_logged_in_handler)